In [1]:
import pandas as pd
df = pd.read_csv("inference_data_indices.csv")
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')

In [2]:
sitka_spruce = pd.read_csv("Sitka spruce.csv", delimiter='\t')
sitka_spruce = sitka_spruce[sitka_spruce["occurrenceStatus"] == "PRESENT"]
sitka_spruce = sitka_spruce[sitka_spruce['year']>=2022]
sitka_spruce['eventDate'] = pd.to_datetime(sitka_spruce['eventDate'], format='%Y-%m-%d', errors='coerce')
sitka_spruce = sitka_spruce[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
sitka_spruce = sitka_spruce.dropna().reset_index(drop=True)

grey_squirrel = pd.read_csv("grey squirrel.csv", delimiter='\t')
grey_squirrel = grey_squirrel[grey_squirrel["occurrenceStatus"] == "PRESENT"]
grey_squirrel = grey_squirrel[grey_squirrel['year']>=2022]
grey_squirrel['eventDate'] = pd.to_datetime(grey_squirrel['eventDate'], format='%Y-%m-%d', errors='coerce')
grey_squirrel = grey_squirrel[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
grey_squirrel = grey_squirrel.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_80939/983191898.py:1: DtypeWarning: Columns (41,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  sitka_spruce = pd.read_csv("Sitka spruce.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_80939/983191898.py:8: DtypeWarning: Columns (2,10,36,37,38,39,40,41,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  grey_squirrel = pd.read_csv("grey squirrel.csv", delimiter='\t')


In [4]:
import numpy as np
from sklearn.neighbors import BallTree
from tqdm import tqdm

# Convert lat/lon to radians for BallTree
red_squirrel_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
sitka_spruce_coords = np.deg2rad(sitka_spruce[['decimalLatitude', 'decimalLongitude']].values)
grey_squirrel_coords = np.deg2rad(grey_squirrel[['decimalLatitude', 'decimalLongitude']].values)


# Build BallTree using haversine metric
tree_sitka_spruce = BallTree(sitka_spruce_coords, metric='haversine')
tree_grey_squirrel = BallTree(grey_squirrel_coords, metric='haversine')

# Define search radius: 1 km in radians
earth_radius_km = 6371.0
radius = 1 / earth_radius_km  # 1 km in radians

# Initialize presence columns
df['sitka_spruce_presence'] = 0
df['grey_squirrel_presence'] = 0

# Iterate through each red squirrel point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = red_squirrel_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # SITKA SPRUCE
    idxs = tree_sitka_spruce.query_radius(point, r=radius)[0]
    for j in idxs:
        sitka_date = sitka_spruce.loc[j, 'eventDate']
        if sitka_date.month == event_month and sitka_date.year == event_year:
            df.at[i, 'sitka_spruce_presence'] = 1
            break  # Found at least one match, no need to check further

    # GREY SQUIRREL
    idxs = tree_grey_squirrel.query_radius(point, r=radius)[0]
    for j in idxs:
        grey_squirrel_date = grey_squirrel.loc[j, 'eventDate']
        if grey_squirrel_date.month == event_month and grey_squirrel_date.year == event_year:
            df.at[i, 'grey_squirrel_presence'] = 1
            break

100%|██████████| 244359/244359 [00:51<00:00, 4759.96it/s]


In [ ]:
from pyproj import Transformer
from rasterio.windows import Window
import rasterio

land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(df['decimalLongitude'], df['decimalLatitude']))
df['easting_ni'], df['northing_ni'] = zip(*transformer_ni.itransform(coords))
df['easting_gb'], df['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class(row):
    try:
        # Try GB raster first
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])
            # Read a small window around the point for better performance
            window = Window(col_idx, row_idx, 1, 1)
            land_cover_class = src.read(1, window=window)[0, 0]

            if land_cover_class == 0:  # Check NI raster if GB is 0
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx, col_idx = src_ni.index(row['easting_ni'], row['northing_ni'])
                    window = Window(col_idx, row_idx, 1, 1)
                    land_cover_class = src_ni.read(1, window=window)[0, 0]

        return land_cover_map.get(land_cover_class, "Unknown")
    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


tqdm.pandas()
df['Land_cover'] = df.progress_apply(get_land_cover_class, axis=1)


  0%|          | 1094/244359 [00:06<18:02, 224.77it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0


  1%|          | 1504/244359 [00:07<16:03, 252.12it/s]

In [ ]:
df = df.dropna()
df=df.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
df = df[df['Land_cover'] != "Unknown"].reset_index(drop=True)
df.to_csv("red_squirrel_inference_data_preprocessed.csv", index=False)